In [1]:
%cd ..
from langchain.chains.combine_documents import collapse_docs, split_list_of_docs
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import Document, StrOutputParser
from langchain_community.vectorstores import FAISS
import faiss
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import json
import re
from random import seed,shuffle
from time import sleep

/home/dp/hdd/gits/hollywood-hooah/model


In [2]:
import pandas as pd


In [3]:
from ragchat.doc_store import DocStore
from ragchat.configs import (DEBUG, REFERENCE_FOLDER, DB_NAME, QUESTIONS_VECTOR_STORE_SAVE_PATH,
COLLECTION_NAME, Q_DB_NAME, Q_COLLECTION_NAME, TOKENS_PER_SYNTH_QUESTION)
from ragchat.text_embedder import TextEmbedder


In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)


In [5]:
vs = FAISS.load_local('vector_stores/faiss_index_0c7394a1b7ff35a035b5adbc4dbb8fae',OpenAIEmbeddings(model="text-embedding-ada-002"))


In [6]:
vs.__dict__.keys()

dict_keys(['embedding_function', 'index', 'docstore', 'index_to_docstore_id', 'distance_strategy', 'override_relevance_score_fn', '_normalize_L2'])

In [7]:
num_docs = vs.index.ntotal
# Get the dimension of your embeddings
embedding_dimension = vs.index.d

embedding_arr = faiss.rev_swig_ptr(vs.index.get_xb(), num_docs*embedding_dimension).reshape(num_docs, embedding_dimension)
embedding_arr.shape

(14712, 1536)

In [16]:

pg=[];pg_num=[];neither=[]
for key,c in vs.docstore.__dict__['_dict'].items():
    if 'page' in c.metadata:
        pg.append(c)
    elif 'page_number' in c.metadata:
        pg_num.append(c)
    else:
        neither.append(c)
    
    


In [17]:
print(len(pg),len(pg_num),len(neither))

7356 7356 0
